<table style="border: none" align="center">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="4" color="black"><b>Network Intrusion Detection</b></font></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr> 
   <tr style="border: none">
       <td style="border: none"><img src="https://github.com/pmservice/wml-sample-models/raw/master/tensorflow/hand-written-digit-recognition/images/experiment_banner.png" width="600" height = "200" alt="Icon"></td>
   </tr>
</table>

This notebook contains steps and code to use Spark ML library to build classification models using kddcup data.

**Notebook environment:** Scala 2.11 + Spark 2.2

**Platform:** IBM Watson Studio

**Dataset:**
[UCI kddcup data](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html) (743MB)

**Purpose:**
Build algorithms that can detect the network intrusions.

**Classification algorithms:**
- Random Forest Classifier
- Multilayer Perceptron Classifier

**Contents: **

This notebook contains the following parts:

1.	[Download data](#download)
2.	[Load and prepare data](#load)
3.	[Building models](#model)
  * [Random Forest](#rf)
  * [MLP](#mlp)


<a id="download"></a>
## Download data
Watson Studio's community has many different data sources. We can download the data from the community. In community, search "KDD" and open the "KDDCUP" dataset, click on the link button to get the URL.

We firstly download the zipped file to Watson's shared directory */opt/ibm/user-libs/shared-data*. If the "shared-data" folder doesn't exist, try to execute the commented code to create the folder.

In [ ]:
import urllib.request 

# !mkdir /opt/ibm/user-libs/shared-data/
url = "https://dataplatform.ibm.com/exchange-api/v1/entries/1438a61212a64ac435c837ba046efc19/data?accessKey=903188bb984a30f38bb889102a7db39f"
filename = "/opt/ibm/user-libs/shared-data/kddcup.zip"
urllib.request.urlretrieve(url, filename)

**Unzip**

In [ ]:
!mkdir /opt/ibm/user-libs/shared-data/kddcup
!unzip /opt/ibm/user-libs/shared-data/kddcup.zip -d /opt/ibm/user-libs/shared-data/kddcup/

In [ ]:
!ls /opt/ibm/user-libs/shared-data/kddcup/

**We use the entire dataset kddcup.data (743MB) , use *gunzip* to unzip the file to the same directory.**

In [ ]:
!gunzip /opt/ibm/user-libs/shared-data/kddcup/kddcup.data.gz -d /opt/ibm/user-libs/shared-data/kddcup/kddcup.data

<a id="load"></a>
## Load and prepare data
The data is comma splited, so we can directly use SparkSession to read the data as dataframe

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.read\
  .format('csv')\
  .option("inferSchema", "true")\
  .load("/opt/ibm/user-libs/shared-data/kddcup/kddcup.data")
df.show(5)

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20180622194457-0001
+---+---+----+---+---+-----+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-------+
|_c0|_c1| _c2|_c3|_c4|  _c5|_c6|_c7|_c8|_c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|_c24|_c25|_c26|_c27|_c28|_c29|_c30|_c31|_c32|_c33|_c34|_c35|_c36|_c37|_c38|_c39|_c40|   _c41|
+---+---+----+---+---+-----+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-------+
|  0|tcp|http| SF|215|45076|  0|  0|  0|  0|   0|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   1|   1| 0.0| 0.0| 0.0| 0.0| 1.0| 0.0| 0.0|   0|   0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|normal.|
|  0|tcp|http| SF|162| 4528|  0|  0|  0|  0|   0|   1|   0|  

**Let's take a look at the schema and labels(the last column"_c41").**

In [2]:
df.printSchema

<bound method DataFrame.printSchema of DataFrame[_c0: int, _c1: string, _c2: string, _c3: string, _c4: int, _c5: int, _c6: int, _c7: int, _c8: int, _c9: int, _c10: int, _c11: int, _c12: int, _c13: int, _c14: int, _c15: int, _c16: int, _c17: int, _c18: int, _c19: int, _c20: int, _c21: int, _c22: int, _c23: int, _c24: double, _c25: double, _c26: double, _c27: double, _c28: double, _c29: double, _c30: double, _c31: int, _c32: int, _c33: double, _c34: double, _c35: double, _c36: double, _c37: double, _c38: double, _c39: double, _c40: double, _c41: string]>

In [3]:
df.select("_c41").groupBy("_c41").count().show()

+----------------+-------+
|            _c41|  count|
+----------------+-------+
|    warezmaster.|     20|
|          smurf.|2807886|
|            pod.|    264|
|           imap.|     12|
|           nmap.|   2316|
|   guess_passwd.|     53|
|        ipsweep.|  12481|
|      portsweep.|  10413|
|          satan.|  15892|
|           land.|     21|
|     loadmodule.|      9|
|      ftp_write.|      8|
|buffer_overflow.|     30|
|        rootkit.|     10|
|    warezclient.|   1020|
|       teardrop.|    979|
|           perl.|      3|
|            phf.|      4|
|       multihop.|      7|
|        neptune.|1072017|
+----------------+-------+
only showing top 20 rows



**According to the [description](http://kdd.ics.uci.edu/databases/kddcup99/training_attack_types), we should recode the labels into five categories. We can use a SQL query to do this. Name the new column name as "label_s" stands for *label in string*.**

In [4]:
df.createOrReplaceTempView("attack")
query = """SELECT *, 
    CASE _c41 
        WHEN 'back.' THEN 'dos'
        WHEN 'buffer_overflow.' THEN 'u2r'
        WHEN 'ftp_write.' THEN 'r2l'
        WHEN 'guess_passwd.' THEN 'r2l'
        WHEN 'imap.' THEN 'r2l'
        WHEN 'ipsweep.' THEN 'probe'
        WHEN 'land.' THEN 'dos'
        WHEN 'loadmodule.' THEN 'u2r'
        WHEN 'multihop.' THEN 'r2l'
        WHEN 'neptune.' THEN 'dos'
        WHEN 'nmap.' THEN 'probe'
        WHEN 'perl.' THEN 'u2r'
        WHEN 'phf.' THEN 'r2l'
        WHEN 'pod.' THEN 'dos'
        WHEN 'portsweep.' THEN 'probe'
        WHEN 'rootkit.' THEN 'u2r'
        WHEN 'satan.' THEN 'probe'
        WHEN 'smurf.' THEN 'dos'
        WHEN 'spy.' THEN 'r2l'
        WHEN 'teardrop.' THEN 'dos'
        WHEN 'warezclient.' THEN 'r2l'
        WHEN 'warezmaster.' THEN 'r2l'
        ELSE 'normal'
END AS label_s 
FROM attack"""

labeled = spark.sql(query)
labeled.select("label_s").groupBy("label_s").count().show()

+-------+-------+
|label_s|  count|
+-------+-------+
|    u2r|     52|
| normal| 972781|
|    r2l|   1126|
|  probe|  41102|
|    dos|3883370|
+-------+-------+



**Intuitively, we should split the data into *training* and *testing* sets before building ML pipeline. However, the number of categories for those categorical variables may be different between two sets. It will cause errors in building algorithms.**

Therefore, we build a pipeline to prepare the data before splitting it to avoid errors.

**The pipeline:**
* StringIndexers: the c1, c2, c3 are categorical strings, we need to firstly index them
* OneHotEncoders: after the categorical strings are indexed, we can now perform one-hot encoding to the indexed columns
* VectorAssembler: Include the wanted columns and assemble them as a feature vector
* labelIndexer: another StringIndexer to index the label_s column and output as "label" column

In [5]:
from pyspark.ml.feature import IndexToString, StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml import Pipeline
indexer1 = StringIndexer(inputCol="_c1", outputCol="i_c1")
indexer2 = StringIndexer(inputCol="_c2", outputCol="i_c2")
indexer3 = StringIndexer(inputCol="_c3", outputCol="i_c3")

encoder1 = OneHotEncoder(inputCol="i_c1", outputCol="v_c1")
encoder2 = OneHotEncoder(inputCol="i_c2", outputCol="v_c2")
encoder3 = OneHotEncoder(inputCol="i_c3", outputCol="v_c3")

featurenames = ["_c0", "v_c1", "v_c2", "v_c3", "_c4", "_c5", "_c6", 
                         "_c7", "_c8", "_c9", "_c10", "_c11", "_c12", "_c13", 
                         "_c14", "_c15", "_c16", "_c17", "_c18", "_c19",
                         "_c22", "_c23", "_c24", "_c25", "_c26", "_c27", 
                         "_c28", "_c29", "_c30", "_c31", "_c32", "_c33", "_c34", 
                         "_c35", "_c36", "_c37", "_c38", "_c39", "_c40"]
assembler = VectorAssembler(inputCols=featurenames, outputCol="features")

labelIndexer = StringIndexer(inputCol="label_s", outputCol="label")

pipeline_prepare = Pipeline(stages=[indexer1,indexer2,indexer3,encoder1,encoder2,encoder3,assembler,labelIndexer])


**Fit and transform the data**

In [6]:
prepare = pipeline_prepare.fit(labeled)
data = prepare.transform(labeled)

<a id="build"></a>
## Building models
**Firstly we split the data, because the data is rather big, I choose 60% of the data to be training set and the rest goes to testing set**

In [7]:
(train, test) = data.randomSplit([0.6, 0.4])

<a id="rf"></a>
### Random Forest
Let's start with random forest algorithm. Spark ML provides this algorithm and the only thing we need to do it set it up. One thing we need to notice is that there are 70 categories in c2 column, so the default _MaxBins_ is not enough (it has to be larger than the biggest number of categories of all categorical variables). We set the _MaxBins_ to be 72 to avoid errors. 

In [8]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=5, maxBins=72)

**Train and fit the model to test data**

In [9]:
import time
start_time = time.time()
rf_model = rf.fit(train)
print("Training process takes %s secs" % (time.time() - start_time))

Training process takes 380.5171010494232 secs


In [10]:
print(rf_model.toDebugString)

RandomForestClassificationModel (uid=RandomForestClassifier_494480e853acbfe78808) with 5 trees
  Tree 0 (weight 1.0):
    If (feature 89 <= 0.0)
     If (feature 8 in {0.0})
      If (feature 98 <= 8.0)
       If (feature 112 <= 0.4)
        If (feature 109 <= 0.04)
         Predict: 1.0
        Else (feature 109 > 0.04)
         Predict: 1.0
       Else (feature 112 > 0.4)
        If (feature 72 in {0.0})
         Predict: 2.0
        Else (feature 72 not in {0.0})
         Predict: 2.0
      Else (feature 98 > 8.0)
       If (feature 105 <= 0.5)
        If (feature 75 in {0.0})
         Predict: 0.0
        Else (feature 75 not in {0.0})
         Predict: 2.0
       Else (feature 105 > 0.5)
        If (feature 7 in {0.0})
         Predict: 2.0
        Else (feature 7 not in {0.0})
         Predict: 2.0
     Else (feature 8 not in {0.0})
      If (feature 0 <= 0.0)
       If (feature 105 <= 0.11)
        Predict: 1.0
       Else (feature 105 > 0.11)
        If (feature 107 <= 254.0)
 

In [11]:
rf_prediction = rf_model.transform(test)

**Check the error and accuracy (The model is actually pretty good)**

In [12]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
rf_accuracy = evaluator.evaluate(rf_prediction)

In [13]:
print("Test Error of RF = %g " % (1.0 - rf_accuracy))

Test Error of RF = 0.00370063 


<a id="mlp"></a>
### Multilayer Perceptron Classifier
Before building the MLP model, we need to know how many nodes are needed for the input layer. Check the feature vector:

In [14]:
train.select("features").show(5)

+--------------------+
|            features|
+--------------------+
|(117,[1,10,72,82,...|
|(117,[1,10,72,82,...|
|(117,[1,10,72,82,...|
|(117,[1,10,72,82,...|
|(117,[1,10,72,82,...|
+--------------------+
only showing top 5 rows



**The input layer should have 117 nodes, the output layer should have 5 nodes (5 label categories). I add one hidden layer with 10 nodes to build the model**

In [15]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
layers = [117, 10, 5]
mlp = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)
start_time = time.time()
mlp_model = mlp.fit(train)
print("Training process takes %s secs" % (time.time() - start_time))

Training process takes 874.324070930481 secs


**Check model performance**

In [16]:
mlp_predictions = mlp_model.transform(test)
accuracy = evaluator.evaluate(mlp_predictions)
print("Test Error of MLP = " + (1.0 - accuracy))

TypeError: Can't convert 'float' object to str implicitly

<a id="summary"></a>
## Summary and next steps     
**Two well-performing models are built in this notebook. It is very easy to build models using Spark API!**

**There is no need to configure the Spark environment in Watson Studio. Just provision the Spark environment, create the notebook and you are ready to write your code!**

**The speed of the Spark enviornment is not bad. However it is much faster if coding in Scala(Check my another demo on using Scala to build the exact same models with the same runtime).**

Next steps:
* Save/download the models to local
* Save and deploy the models using Watson Machine Learning Service(WML)
